## main

### Classical Model Architecture:
1. **Input Layer:**
   - Input size: **28x28**
   - The input is a grayscale image (1 channel).

2. **Convolutional Block 1:**
   - **Convolutional layer:**
     - Input channels: **1**
     - Output channels: **16**
     - Kernel size: **5x5**
     - Stride: **1**
     - Padding: **2 pixels**
   - **Batch Normalization**: Applied to the output of the convolutional layer.
   - **Activation Function**: **ReLU**.
   - **Max Pooling**: Kernel size: **2x2**
     - Feature map dimension after Max Pooling: **16x14x14**.

3. **Convolutional Block 2:**
   - **Convolutional layer:**
     - Input channels: **16**
     - Output channels: **32**
     - Kernel size: **5x5**
     - Stride: **1**
     - Padding: **2 pixels**
   - **Batch Normalization**: Applied to the output of the convolutional layer.
   - **Activation Function**: **ReLU**.
   - **Max Pooling**: Kernel size: **2x2**
     - Feature map dimension after Max Pooling: **32x7x7**.

4. **Flatten Layer:**
   - **Output dimension:** Flatten the feature map to **1568 $\times$ n**, where $ n $ is the number of feature vectors corresponding to the batch size.

5. **Fully Connected Layer:**
   - The fully connected layer bridges the classical and quantum layers.
   - **Input:** 1568 features.
   - **Output:** Direct input into the quantum layer.


---

## **Structure of the Quantum Layer**

### **Overview:**

The quantum component consists of **$c$ parallel quantum layers** (Parameterized Quantum Circuits, PQCs). Each PQC is composed of three main parts:

1. **Embedding** (Encoding classical data into quantum states)
2. **Variational gates** (Trainable quantum transformations)
3. **Measurement** (Extracting classical output from quantum states)

---

### **Step-by-Step Breakdown:**

#### **1. Input Data Division:**

* The **$n$ features** from the previous classical fully connected layer are divided into **$c$ parts**.
* Each part contains **$q$ values** (where $n = c \times q$).
* Each part is a vector: $\mathbf{x} = (\phi_1, \phi_2, ..., \phi_q) \in \mathbb{R}^q$.
* **Important:** $n$ must be divisible by $q$, so that $c = n/q$.

**Example:**

* If $n = 100$ and $q = 5$, then $c = 20$ parallel quantum circuits.
* Each circuit processes a vector of 5 features.

---

#### **2. Encoding (Embedding):**

* The classical features are encoded into quantum states using **"angle embedding"**.
* **Angle embedding** rotates each qubit (starting from the ground state $|0\rangle$) around the **X-axis** on the Bloch sphere by an angle proportional to the corresponding feature value.
* Mathematically:

$$
|\psi\rangle = R_x^{\text{emb}}(\mathbf{x}) |\psi_0\rangle
$$

where $|\psi_0\rangle = |0\rangle^{\otimes q}$ (all qubits start in the ground state).

* For each qubit $j$, the rotation angle is $\phi_j$:

$$
R_x(\phi_j) =
\begin{pmatrix}
\cos(\phi_j/2) & -i\sin(\phi_j/2) \
-i\sin(\phi_j/2) & \cos(\phi_j/2)
\end{pmatrix}
$$

**What this does:**

* Converts the classical vector $\mathbf{x} = (\phi_1, \phi_2, ..., \phi_q)$ into a quantum state that encodes the input data.

---

#### **3. Variational Part (Trainable Gates):**

The variational part consists of **two components**:

##### **a. Rotations with Trainable Parameters:**

* Each qubit undergoes rotations (e.g., $R_x$, $R_y$, $R_z$) controlled by **trainable parameters** $\theta$.
* These rotations transform the encoded quantum state according to the variational parameters.

##### **b. CNOT Operations (Entanglement):**

* **CNOT gates** are applied to entangle the qubits within the PQC.
* Entanglement allows qubits to share information, enabling the quantum circuit to capture complex patterns in the data.

##### **Depth of the Variational Part:**

* The variational part is repeated $i$ times, where $i$ is a **hyperparameter** (the depth/number of iterations).
* Each iteration applies rotations followed by CNOT operations.

**Important:**

* The variational parameters $\theta$ are **different** for each of the $i$ repetitions and for each of the $c$ quantum circuits.
* **Total number of trainable parameters (weights)** in the quantum layer: $q \cdot 3i \cdot c$.

---

#### **4. Measurement:**

* After the variational operations, **measurement** is performed in the **Pauli-Y basis** for each qubit.
* The measurement result for the $j$-th PQC is:

$$
v^{(j)} =
\langle 0 |
R_x^{\text{emb}}(\phi_j)^\dagger
U(\theta)^\dagger
Y_j
U(\theta)
R_x^{\text{emb}}(\phi_j)
| 0 \rangle
$$

where:

* $Y_j$ is the **Pauli-Y matrix** for the $j$-th qubit.

* $R_x^{\text{emb}}(\phi_j)$ is the embedding operation.

* $U(\theta)$ represents the variational gates.

* $\theta$ is the vector of trainable parameters.

* After this operation, we obtain a vector $\mathbf{v} \in \mathbb{R}^q$ for each PQC.

---

#### **5. Output Concatenation:**

* The outputs of all $c$ PQCs are **concatenated** to form a new vector:

$$
\hat{\mathbf{v}} \in \mathbb{R}^n
$$

* This vector $\hat{\mathbf{v}}$ serves as the input to the **second classical fully connected layer**.

---

### **Summary of the Quantum Layer:**

| **Component**         | **Description**                                                                         |
| --------------------- | --------------------------------------------------------------------------------------- |
| **Input**             | $n$ features from the classical layer, divided into $c$ parts of $q$ values each.       |
| **Encoding**          | Angle embedding using $R_x$ rotations on each qubit.                                    |
| **Variational Gates** | Rotations ($R_x$, $R_y$, $R_z$) with trainable parameters $\theta$, repeated $i$ times. |
| **Entanglement**      | CNOT gates applied between qubits to entangle them.                                     |
| **Measurement**       | Pauli-Y basis measurement to extract classical output.                                  |
| **Output**            | $n$ features (concatenated from all $c$ PQCs) passed to the next classical layer.       |

---

### **Key Hyperparameters:**

| **Parameter** | **Meaning**                                                               |
| ------------- | ------------------------------------------------------------------------- |
| $n$           | Total number of features (input/output size of the quantum layer).        |
| $q$           | Number of qubits per PQC (must divide $n$).                               |
| $c$           | Number of parallel quantum circuits ($c = n/q$).                          |
| $i$           | Depth of the variational part (number of repetitions of rotation + CNOT). |

---

### **Total Number of Trainable Parameters:**

$$
\text{Total weights} = q \cdot 3i \cdot c
$$

* $q$: Number of qubits.
* $3i$: Each qubit has 3 rotation gates ($R_x$, $R_y$, $R_z$) repeated $i$ times.
* $c$: Number of quantum circuits.



In [ ]:
!pip install pennylane pennylane-lightning torch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pennylane as qml
import numpy as np
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:

class QuantumLayer(nn.Module):
    def __init__(self, n_features, n_qubits, n_circuits, depth, device='default.qubit'):
        """
        Quantum Layer for HQNN-Parallel
        
        Parameters:
        - n_features (n): Total number of input/output features
        - n_qubits (q): Number of qubits per quantum circuit
        - n_circuits (c): Number of parallel quantum circuits (c = n / q)
        - depth (i): Depth of variational circuit (number of repetitions)
        - device: PennyLane device (default: 'default.qubit' for simulation)
        """
        super(QuantumLayer, self).__init__()
        
        # Hyperparameters
        self.n_features = n_features
        self.n_qubits = n_qubits
        self.n_circuits = n_circuits
        self.depth = depth
        
        # Validate that n is divisible by q
        assert n_features == n_qubits * n_circuits, \
            f"n_features ({n_features}) must equal n_qubits ({n_qubits}) * n_circuits ({n_circuits})"
        
        # Create quantum devices (one per circuit)
        self.devices = [qml.device(device, wires=n_qubits) for _ in range(n_circuits)]
        
        # Create QNodes (quantum circuits)
        self.qnodes = [qml.QNode(self._circuit, dev) for dev in self.devices]
        
        # Initialize trainable parameters
        # Shape: (n_circuits, depth, n_qubits, 3) for Rx, Ry, Rz rotations
        self.weights = nn.Parameter(
            torch.randn(n_circuits, depth, n_qubits, 3) * 0.01
        )
        
    def _circuit(self, inputs, weights):
        """
        Single Parameterized Quantum Circuit (PQC)
        
        Parameters:
        - inputs: Vector of q classical features (φ₁, φ₂, ..., φ_q)
        - weights: Trainable parameters for this circuit
        
        Returns:
        - Expectation values of Pauli-Y measurements for all qubits
        """
        # 1. EMBEDDING: Angle embedding using Rx rotations
        for i in range(self.n_qubits):
            qml.RX(inputs[i], wires=i)
        
        # 2. VARIATIONAL PART: Repeated layers of rotations + entanglement
        for layer in range(self.depth):
            # Apply trainable rotations (Rx, Ry, Rz) to each qubit
            for i in range(self.n_qubits):
                qml.RX(weights[layer, i, 0], wires=i)
                qml.RY(weights[layer, i, 1], wires=i)
                qml.RZ(weights[layer, i, 2], wires=i)
            
            # Apply CNOT gates for entanglement (circular pattern)
            for i in range(self.n_qubits - 1):
                qml.CNOT(wires=[i, i + 1])
            # Close the loop (optional, creates more entanglement)
            if self.n_qubits > 1:
                qml.CNOT(wires=[self.n_qubits - 1, 0])
        
        # 3. MEASUREMENT: Measure in Pauli-Y basis for all qubits
        return [qml.expval(qml.PauliY(i)) for i in range(self.n_qubits)]
    
    def forward(self, x):
        """
        Forward pass through the quantum layer
        
        Parameters:
        - x: Input tensor of shape (batch_size, n_features)
        
        Returns:
        - Output tensor of shape (batch_size, n_features)
        """
        batch_size = x.shape[0]
        outputs = []
        
        # Process each sample in the batch
        for sample_idx in range(batch_size):
            sample = x[sample_idx]
            circuit_outputs = []
            
            # Process each of the c quantum circuits in parallel
            for circuit_idx in range(self.n_circuits):
                # Extract q features for this circuit
                start_idx = circuit_idx * self.n_qubits
                end_idx = start_idx + self.n_qubits
                circuit_input = sample[start_idx:end_idx]
                
                # Get weights for this circuit
                circuit_weights = self.weights[circuit_idx]
                
                # Execute quantum circuit
                result = self.qnodes[circuit_idx](circuit_input, circuit_weights)
                circuit_outputs.extend(result)
            
            outputs.append(circuit_outputs)
        
        # Convert to tensor
        return torch.stack([torch.tensor(out, dtype=torch.float32) for out in outputs])


: 

In [ ]:
# Hyperparameters
n = 100      # Total features
q = 5        # Qubits per circuit
c = 20       # Number of circuits (n/q)
i = 3        # Depth of variational circuit
batch_size = 4

# Create quantum layer
quantum_layer = QuantumLayer(n_features=n, n_qubits=q, n_circuits=c, depth=i)

# Test with random input
test_input = torch.randn(batch_size, n)
print(f"Input shape: {test_input.shape}")

# Forward pass
output = quantum_layer(test_input)
print(f"Output shape: {output.shape}")
print(f"Number of trainable parameters: {sum(p.numel() for p in quantum_layer.parameters())}")

# Verify parameter count: q * 3 * i * c
expected_params = q * 3 * i * c
print(f"Expected parameters: {expected_params}")

In [ ]:
class HybridQuantumClassicalModel(nn.Module):
    def __init__(self, input_channels=1, n=100, q=5, depth=3, num_classes=10):
        """
        Complete Hybrid Quantum-Classical Neural Network (HQNN-Parallel)
        
        Parameters:
        - input_channels: Number of input channels (1 for grayscale)
        - n: Number of features for quantum layer
        - q: Number of qubits per quantum circuit
        - depth: Depth of variational circuit
        - num_classes: Number of output classes
        """
        super(HybridQuantumClassicalModel, self).__init__()
        
        # Calculate number of circuits
        assert n % q == 0, "n must be divisible by q"
        c = n // q
        
        # Store hyperparameters
        self.n = n
        self.q = q
        self.c = c
        self.depth = depth
        self.num_classes = num_classes
        
        # ==================== Classical Convolutional Part ====================
        self.conv1 = nn.Conv2d(
            in_channels=input_channels, 
            out_channels=16, 
            kernel_size=5, 
            stride=1, 
            padding=2
        )
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(
            in_channels=16, 
            out_channels=32, 
            kernel_size=5, 
            stride=1, 
            padding=2
        )
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        # ==================== Hybrid Dense Layers ====================
        # First FC layer (Classical -> Quantum interface)
        self.fc1 = nn.Linear(32 * 7 * 7, n)  # 1568 -> n
        self.bn_fc1 = nn.BatchNorm1d(n)
        
        # Quantum Layer
        self.quantum_layer = QuantumLayer(n_features=n, n_qubits=q, n_circuits=c, depth=depth)
        
        # Second FC layer (Quantum -> Classical interface)
        self.fc2 = nn.Linear(n, num_classes)  # n -> num_classes
        self.bn_fc2 = nn.BatchNorm1d(num_classes)

    def forward(self, x):
        # Convolutional part
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # First dense layer
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = F.relu(x)
        
        # Quantum layer
        x = self.quantum_layer(x)
        
        # Second dense layer
        x = self.fc2(x)
        x = self.bn_fc2(x)
        x = F.relu(x)
        
        return x

    def get_hyperparameters(self):
        return {
            'n': self.n,
            'q': self.q,
            'c': self.c,
            'depth': self.depth,
            'num_classes': self.num_classes,
            'quantum_params': self.q * 3 * self.depth * self.c
        }

In [ ]:
# Hyperparameters
n = 100
q = 5
depth = 3
num_classes = 10

# Create model
model = HybridQuantumClassicalModel(input_channels=1, n=n, q=q, depth=depth, num_classes=num_classes)

print(model)
print("\nHyperparameters:")
print(model.get_hyperparameters())

# Test forward pass
dummy_input = torch.randn(2, 1, 28, 28)  # Small batch for testing
output = model(dummy_input)
print(f"\nOutput shape: {output.shape}")

In [ ]:
# Hyperparameters
n = 20           # Reduced for faster testing
q = 5
depth = 2        # Reduced for faster testing
num_classes = 10
batch_size = 8   # Small batch due to quantum simulation overhead
learning_rate = 0.001
num_epochs = 5

# Data loading
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [ ]:

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model, loss, optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridQuantumClassicalModel(input_channels=1, n=n, q=q, depth=depth, num_classes=num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:

# Training loop
print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if batch_idx % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}] completed. Average Loss: {avg_loss:.4f}')

print("Training completed!")